In [1]:
from bayes_opt import BayesianOptimization
from bayes_opt.event import Events
from bayes_opt.logger import JSONLogger
from numpy import where
from pandas import DataFrame, Series, concat, read_csv
from sklearn.metrics import f1_score
from sklearn.svm import OneClassSVM

logger = JSONLogger(path="../reports/logs.log")

In [2]:
X_train = read_csv("../data/PAMAP2/x_train_data.csv")
X_test = read_csv("../data/PAMAP2/x_test_data.csv")
y_train = read_csv("../data/PAMAP2/y_train_data.csv")
y_test = read_csv("../data/PAMAP2/y_test_data.csv")

X_train["activity"] = y_train  # First 80% of the data
X_test["activity"] = y_test  # Last 20% of the data

models: dict[int, OneClassSVM] = {}
training_data: DataFrame
testing_data: DataFrame
train_targets: Series
test_targets: Series

In [3]:
def objective_function(nu: float, gamma: float) -> float:
    """
    Objective function to optimize F1-Score on the test set.

    Args:
        nu (float): nu param to evaluate.
        gamma (float): gamma param to evaluate.

    Returns:
        float: F1-Score on the test set of this iteration.
    """
    oc_svm = OneClassSVM(kernel="rbf", nu=nu, gamma=gamma).fit(training_data)

    f1_train = f1_score(
        train_targets,
        y_pred_train := where(oc_svm.predict(training_data) == 1, False, True),
        average="macro",
    )
    f1_test = f1_score(
        test_targets,
        y_pred_test := where(oc_svm.predict(testing_data) == 1, False, True),
        average="macro",
    )
    print(
        f"\nF1 Score (Train): {f1_train}"
        f"\nF1 Score (Test): {f1_test}"
        f"\nErrors in Train {y_pred_train[y_pred_train is False].size}"
        f"\nErrors in Test {y_pred_test[y_pred_test is False].size}"
    )
    return float(f1_test)

In [4]:
for activity in X_train["activity"].unique():
    print(f"Activity: {activity}")
    novelty: DataFrame = X_train[X_train["activity"] == activity].copy()
    current_data: DataFrame = X_train[~X_train.index.isin(novelty.index)].copy()

    current_data.loc[:, "isNovelty"], novelty.loc[:, "isNovelty"] = False, True

    training_data = current_data.drop(columns=["isNovelty"])
    train_targets = current_data["isNovelty"]

    sampled_data = current_data.sample(n=int(0.2 * len(current_data)), random_state=42)
    sampled_data["isNovelty"] = 0
    novelty = concat([novelty, sampled_data])

    # only current activity (as novelty)
    testing_data = novelty.drop(columns=["isNovelty"])
    test_targets = novelty["isNovelty"]

    optimizer = BayesianOptimization(
        f=objective_function,
        pbounds={"nu": (0.01, 0.5), "gamma": (1e-4, 1e-1)},
        random_state=42,
    )
    optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
    optimizer.maximize(init_points=5, n_iter=25)

    models[activity] = optimizer.max

Activity: 1
